# Tratamento de Lista Geral de Leads

### Objetivo: Esse pipeline tem como objetivo separar a lista de Leads disponível para disparos e consumo

In [1]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


## Load Leads Dataframe

In [2]:
df = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"campaign_id": "isca-aulassemanais", "conversion_type_id": 1})

In [3]:
## Filter Buyers from Dataframe
df = remove_buyers_from_dataframe(df)

✓ Removidos 1,229 compradores de 102,226 registros (1.2% filtrado)


In [4]:
# garante datetime
df["last_conversion_date"] = pd.to_datetime(df["last_conversion_date"], format="%Y-%m-%d", errors="coerce")

cutoff = pd.Timestamp.now().normalize() - pd.Timedelta(days=7)
df_ultimos_8_dias = df[df["last_conversion_date"] >= cutoff].copy()